# Predicting Heart Disease — Playground Series S6E2

**Approach**: 3-model gradient boosting ensemble (CatBoost + XGBoost + LightGBM) with:
- Original dataset augmentation
- Frequency + target encoding with CV-safe feature engineering
- Clinical domain interaction features
- Multi-seed training for stability
- Optimized weighted rank averaging (no stacking leakage)

In [ ]:
import numpy as np
import pandas as pd
import gc
import os
import warnings
from itertools import combinations

from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from scipy.stats import rankdata
from scipy.optimize import minimize

warnings.filterwarnings("ignore")

N_FOLDS = 5
SEEDS = [42, 123, 2026]
TARGET = "Heart Disease"

print("Setup complete.")

## Load Data

Load competition data and the original UCI Heart Disease dataset for augmentation.

In [ ]:
# ---------- paths (Kaggle kernel vs local via kagglehub) ----------
COMP_DIR = "/kaggle/input/playground-series-s6e2"
ORIG_DIR = "/kaggle/input/heart-disease-dataset"

if not os.path.exists(COMP_DIR):
    import kagglehub
    COMP_DIR = kagglehub.competition_download("playground-series-s6e2")
    ORIG_DIR = kagglehub.dataset_download("johnsmith88/heart-disease-dataset")

train = pd.read_csv(os.path.join(COMP_DIR, "train.csv"))
test  = pd.read_csv(os.path.join(COMP_DIR, "test.csv"))

print(f"Train shape: {train.shape}")
print(f"Test  shape: {test.shape}")
print(f"\nTarget distribution:\n{train[TARGET].value_counts()}")

train[TARGET] = train[TARGET].map({"Absence": 0, "Presence": 1}).astype(np.uint8)
y = train[TARGET].values
test_ids = test["id"].values

# ---------- load original dataset for augmentation ----------
ORIG_COL_MAP = {
    "age": "Age", "sex": "Sex", "cp": "Chest pain type",
    "trestbps": "BP", "chol": "Cholesterol",
    "fbs": "FBS over 120", "restecg": "EKG results",
    "thalach": "Max HR", "exang": "Exercise angina",
    "oldpeak": "ST depression", "slope": "Slope of ST",
    "ca": "Number of vessels fluro", "thal": "Thallium",
    "target": TARGET,
}

try:
    orig_path = os.path.join(ORIG_DIR, "heart.csv")
    orig = pd.read_csv(orig_path)
    orig.rename(columns=ORIG_COL_MAP, inplace=True)
    orig[TARGET] = orig[TARGET].astype(np.uint8)

    # align columns to match competition data (drop id if present)
    keep = [c for c in train.columns if c not in ("id",)]
    orig = orig[[c for c in keep if c in orig.columns]]

    n_before = len(train)
    train = pd.concat([train, orig], ignore_index=True)
    y = train[TARGET].values
    print(f"\nOriginal dataset appended: {len(orig)} rows  |  "
          f"Train grew from {n_before} to {len(train)}")
except Exception as e:
    print(f"Original dataset not available ({e}), proceeding without it.")

X_tr_raw = train.drop(columns=[TARGET, "id"], errors="ignore")
X_te_raw = test.drop(columns=["id"])

cat_cols = [
    "Sex", "Chest pain type", "FBS over 120", "EKG results",
    "Exercise angina", "Slope of ST",
    "Number of vessels fluro", "Thallium",
]
num_cols = ["Age", "BP", "Cholesterol", "Max HR", "ST depression"]

print(f"\nFeatures — categorical: {len(cat_cols)}, numerical: {len(num_cols)}")

## Feature Engineering

Build frequency encodings, target encodings (CV-safe), clinical interaction features, and correlation-based interaction growth.

In [ ]:
# ===================== FREQUENCY ENCODING =====================
def freq_encode(tr, te, cols):
    tr_out = pd.DataFrame(index=tr.index)
    te_out = pd.DataFrame(index=te.index)
    for c in cols:
        freq = tr[c].value_counts(normalize=True)
        tr_out[c + "_freq"] = tr[c].map(freq).fillna(0)
        te_out[c + "_freq"] = te[c].map(freq).fillna(0)
    return tr_out, te_out

tr_freq, te_freq = freq_encode(X_tr_raw, X_te_raw, cat_cols + num_cols)

# ===================== TARGET ENCODING (CV-safe) =====================
skf_te = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

tr_te = pd.DataFrame(0.0, index=X_tr_raw.index, columns=[c + "_te" for c in cat_cols + num_cols])
te_te = pd.DataFrame(index=X_te_raw.index)

global_mean = y.mean()

for c in cat_cols + num_cols:
    col_te = c + "_te"
    for tr_i, val_i in skf_te.split(X_tr_raw, y):
        means = train.iloc[tr_i].groupby(X_tr_raw.iloc[tr_i][c])[TARGET].mean()
        tr_te.iloc[val_i, tr_te.columns.get_loc(col_te)] = (
            X_tr_raw.iloc[val_i][c].map(means).fillna(global_mean).values
        )
    full_means = train.groupby(X_tr_raw[c])[TARGET].mean()
    te_te[col_te] = X_te_raw[c].map(full_means).fillna(global_mean)

# ===================== CLINICAL DOMAIN FEATURES =====================
def add_domain_features(df, prefix=""):
    out = pd.DataFrame(index=df.index)
    eps = 1e-6
    out["ST_x_ExAngina"] = df["ST depression_te"] * df["Exercise angina_te"]
    out["MaxHR_x_Age"] = df["Max HR_te"] * df["Age_te"]
    out["Chol_x_BP"] = df["Cholesterol_te"] * df["BP_te"]
    out["ST_div_MaxHR"] = df["ST depression_te"] / (df["Max HR_te"] + eps)
    out["BP_div_Age"] = df["BP_te"] / (df["Age_te"] + eps)
    out["Vessels_x_Thal"] = df["Number of vessels fluro_te"] * df["Thallium_te"]
    out["ChestPain_x_ST"] = df["Chest pain type_te"] * df["ST depression_te"]
    return out

tr_domain = add_domain_features(tr_te)
te_domain = add_domain_features(te_te)

# ===================== CORRELATION-BASED INTERACTION GROWTH =====================
corr_scores = {}
for a, b in combinations(tr_te.columns, 2):
    corr_scores[(a, b)] = abs(np.corrcoef(tr_te[a], tr_te[b])[0, 1])

top_pairs = sorted(corr_scores, key=corr_scores.get, reverse=True)[:10]

tr_corr = pd.DataFrame(index=tr_te.index)
te_corr = pd.DataFrame(index=te_te.index)
for a, b in top_pairs:
    name = f"{a}_x_{b}"
    tr_corr[name] = tr_te[a] * tr_te[b]
    te_corr[name] = te_te[a] * te_te[b]

# ===================== ASSEMBLE FINAL FEATURE MATRICES =====================
X_train = pd.concat([tr_freq, tr_te, tr_domain, tr_corr], axis=1).fillna(0).astype(np.float32)
X_test  = pd.concat([te_freq, te_te, te_domain, te_corr], axis=1).fillna(0).astype(np.float32)

print(f"Final feature matrix: {X_train.shape[1]} features")
print(f"X_train: {X_train.shape}, X_test: {X_test.shape}")

## Model Definitions

In [ ]:
USE_GPU = os.path.exists("/opt/conda")  # heuristic for Kaggle kernels

def get_catboost_params(seed):
    p = dict(
        iterations=10000,
        learning_rate=0.01,
        depth=3,
        loss_function="Logloss",
        eval_metric="AUC",
        auto_class_weights="Balanced",
        bootstrap_type="Bernoulli",
        subsample=0.85,
        l2_leaf_reg=10,
        random_strength=1.0,
        min_data_in_leaf=50,
        verbose=0,
        random_seed=seed,
        early_stopping_rounds=300,
    )
    if USE_GPU:
        p["task_type"] = "GPU"
    return p


def get_xgb_params(seed):
    p = dict(
        objective="binary:logistic",
        eval_metric="auc",
        learning_rate=0.01,
        max_depth=3,
        subsample=0.85,
        colsample_bytree=0.85,
        reg_alpha=0.05,
        reg_lambda=1.5,
        min_child_weight=50,
        n_estimators=10000,
        random_state=seed,
        n_jobs=-1,
        verbosity=0,
    )
    if USE_GPU:
        p["tree_method"] = "hist"
        p["device"] = "cuda"
    return p


def get_lgb_params(seed):
    p = dict(
        objective="binary",
        metric="auc",
        learning_rate=0.01,
        num_leaves=20,
        max_depth=-1,
        subsample=0.85,
        colsample_bytree=0.85,
        reg_alpha=0.05,
        reg_lambda=1.5,
        min_child_samples=50,
        n_estimators=10000,
        random_state=seed,
        n_jobs=-1,
        verbose=-1,
    )
    if USE_GPU:
        p["device"] = "gpu"
    return p


print(f"GPU mode: {USE_GPU}")

## Training Loop — Multi-Seed 5-Fold CV

Each seed produces OOF and test predictions for all 3 models. Early stopping on the validation fold prevents overfitting.

In [ ]:
def train_fold_models(X_tr, y_tr, X_val, y_val, X_te, seed):
    """Train CatBoost, XGBoost, LightGBM on one fold and return val/test probabilities."""
    results = {}

    # --- CatBoost ---
    cb = CatBoostClassifier(**get_catboost_params(seed))
    cb.fit(X_tr, y_tr, eval_set=(X_val, y_val), use_best_model=True)
    results["cb_val"] = cb.predict_proba(X_val)[:, 1]
    results["cb_te"]  = cb.predict_proba(X_te)[:, 1]

    # --- XGBoost ---
    xgb = XGBClassifier(**get_xgb_params(seed), early_stopping_rounds=300)
    xgb.fit(
        X_tr, y_tr,
        eval_set=[(X_val, y_val)],
        verbose=False,
    )
    results["xgb_val"] = xgb.predict_proba(X_val)[:, 1]
    results["xgb_te"]  = xgb.predict_proba(X_te)[:, 1]

    # --- LightGBM ---
    lgb = LGBMClassifier(**get_lgb_params(seed))
    lgb.fit(
        X_tr, y_tr,
        eval_set=[(X_val, y_val)],
        callbacks=[
            __import__("lightgbm").early_stopping(300, verbose=False),
            __import__("lightgbm").log_evaluation(0),
        ],
    )
    results["lgb_val"] = lgb.predict_proba(X_val)[:, 1]
    results["lgb_te"]  = lgb.predict_proba(X_te)[:, 1]

    del cb, xgb, lgb
    gc.collect()
    return results


# ---------- main training loop across seeds ----------
all_oof = {m: np.zeros(len(X_train)) for m in ("cb", "xgb", "lgb")}
all_test = {m: np.zeros(len(X_test)) for m in ("cb", "xgb", "lgb")}

for seed in SEEDS:
    print(f"\n{'='*60}")
    print(f"  SEED {seed}")
    print(f"{'='*60}")

    skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=seed)

    seed_oof = {m: np.zeros(len(X_train)) for m in ("cb", "xgb", "lgb")}
    seed_test = {m: np.zeros(len(X_test)) for m in ("cb", "xgb", "lgb")}

    for fold, (tr_idx, val_idx) in enumerate(skf.split(X_train, y)):
        print(f"\n  Fold {fold+1}/{N_FOLDS}")

        res = train_fold_models(
            X_train.iloc[tr_idx], y[tr_idx],
            X_train.iloc[val_idx], y[val_idx],
            X_test, seed + fold,
        )

        for m in ("cb", "xgb", "lgb"):
            seed_oof[m][val_idx] = res[f"{m}_val"]
            seed_test[m] += res[f"{m}_te"] / N_FOLDS

        fold_scores = {m: roc_auc_score(y[val_idx], res[f"{m}_val"]) for m in ("cb", "xgb", "lgb")}
        print(f"    CB={fold_scores['cb']:.5f}  XGB={fold_scores['xgb']:.5f}  LGB={fold_scores['lgb']:.5f}")
        gc.collect()

    for m in ("cb", "xgb", "lgb"):
        all_oof[m] += seed_oof[m] / len(SEEDS)
        all_test[m] += seed_test[m] / len(SEEDS)
        print(f"  Seed {seed} — {m.upper()} CV AUC: {roc_auc_score(y, seed_oof[m]):.5f}")

print(f"\n{'='*60}")
print("  AVERAGED ACROSS SEEDS")
print(f"{'='*60}")
for m in ("cb", "xgb", "lgb"):
    print(f"  {m.upper()} OOF AUC: {roc_auc_score(y, all_oof[m]):.5f}")

## Optimized Weighted Rank Averaging

Find optimal blend weights by maximizing AUC on truly out-of-fold predictions. No leakage — OOF predictions were never seen during training of their respective folds.

In [ ]:
# rank-normalize OOF and test predictions
oof_ranks = {}
test_ranks = {}
for m in ("cb", "xgb", "lgb"):
    oof_ranks[m] = rankdata(all_oof[m]) / len(all_oof[m])
    test_ranks[m] = rankdata(all_test[m]) / len(all_test[m])


def neg_auc(weights):
    """Negative AUC for scipy minimizer (weights for cb, xgb, lgb)."""
    w = np.abs(weights)
    w = w / w.sum()
    blend = w[0] * oof_ranks["cb"] + w[1] * oof_ranks["xgb"] + w[2] * oof_ranks["lgb"]
    return -roc_auc_score(y, blend)


# grid search + refinement for robustness
best_score = -1
best_w = np.array([1/3, 1/3, 1/3])

for _ in range(30):
    w0 = np.random.dirichlet(np.ones(3))
    res = minimize(neg_auc, w0, method="Nelder-Mead",
                   options={"maxiter": 2000, "xatol": 1e-8, "fatol": 1e-8})
    w = np.abs(res.x)
    w = w / w.sum()
    score = -res.fun
    if score > best_score:
        best_score = score
        best_w = w

print(f"Optimal weights — CB: {best_w[0]:.4f}  XGB: {best_w[1]:.4f}  LGB: {best_w[2]:.4f}")
print(f"Optimized blend CV AUC: {best_score:.6f}")

# equal-weight baseline for comparison
equal_blend = (oof_ranks["cb"] + oof_ranks["xgb"] + oof_ranks["lgb"]) / 3
print(f"Equal-weight  CV AUC:   {roc_auc_score(y, equal_blend):.6f}")

## Generate Submission

In [ ]:
# apply optimal weights to test rank predictions
test_pred = (
    best_w[0] * test_ranks["cb"]
    + best_w[1] * test_ranks["xgb"]
    + best_w[2] * test_ranks["lgb"]
)

submission = pd.DataFrame({"id": test_ids, TARGET: test_pred})
submission.to_csv("submission.csv", index=False)

print(f"Submission shape: {submission.shape}")
print(f"Prediction range: [{test_pred.min():.6f}, {test_pred.max():.6f}]")
print(submission.head(10))
print("\nSubmission saved to submission.csv")